In [78]:
import pandas as pd
import sqlite3

patients = pd.read_csv("/kaggle/input/day19-datasets/day_19_patients.csv")
appointments = pd.read_csv("/kaggle/input/day19-datasets/day_19_appointments.csv")

conn = sqlite3.connect(":memory:")

patients.to_sql("patients", conn, index=False, if_exists="replace")
appointments.to_sql("appointments", conn, index=False, if_exists="replace")


15

In [79]:
pd.read_sql("SELECT * FROM patients", conn)

,patient_id,name,city,age
0,1,Isha,Delhi,24
1,2,Aman,Mumbai,29
2,3,Riya,Pune,22
3,4,Kabir,Delhi,35
4,5,Neha,Bangalore,28
5,6,Saurav,Chennai,31
6,7,Pallavi,Mumbai,26
7,8,Rohit,Pune,27


In [80]:
pd.read_sql("SELECT * FROM appointments", conn)

,appointment_id,patient_id,specialty,fee,appointment_date
0,501,1,General,500,2024-01-05
1,502,1,Dermatology,1200,2024-02-10
2,503,2,Orthopedic,1500,2024-01-12
3,504,2,General,600,2024-03-01
4,505,3,Dental,800,2024-02-03
5,506,3,Dental,900,2024-03-20
6,507,4,Cardiology,2500,2024-03-01
7,508,5,General,500,2024-01-10
8,509,5,ENT,700,2024-02-15
9,510,6,Orthopedic,1800,2024-03-15


# 🟢 LEVEL 1 — Warm-up (Confidence building)

**Show all patients and their cities.**

In [81]:
pd.read_sql("""
SELECT name, city
FROM patients ;
""", conn)


,name,city
0,Isha,Delhi
1,Aman,Mumbai
2,Riya,Pune
3,Kabir,Delhi
4,Neha,Bangalore
5,Saurav,Chennai
6,Pallavi,Mumbai
7,Rohit,Pune


**Show all appointments where the fee is greater than 1000.**

In [82]:
pd.read_sql("""
SELECT *
FROM appointments
WHERE fee > 1000;

""", conn)


,appointment_id,patient_id,specialty,fee,appointment_date
0,502,1,Dermatology,1200,2024-02-10
1,503,2,Orthopedic,1500,2024-01-12
2,507,4,Cardiology,2500,2024-03-01
3,510,6,Orthopedic,1800,2024-03-15
4,512,8,Dermatology,1300,2024-02-22
5,514,4,Cardiology,2700,2024-04-01


**Show patients who live in Delhi.**

In [83]:
pd.read_sql("""
SELECT *
FROM patients
WHERE city = 'Delhi';

""", conn)


,patient_id,name,city,age
0,1,Isha,Delhi,24
1,4,Kabir,Delhi,35


**Show all appointments that happened in March.**

In [84]:
pd.read_sql("""
SELECT *
FROM appointments
WHERE appointment_date LIKE '2024-03%';

""", conn)


,appointment_id,patient_id,specialty,fee,appointment_date
0,504,2,General,600,2024-03-01
1,506,3,Dental,900,2024-03-20
2,507,4,Cardiology,2500,2024-03-01
3,510,6,Orthopedic,1800,2024-03-15
4,513,8,General,500,2024-03-05


# 🟡 LEVEL 2 — JOIN + GROUP BY (Core analytics)

**Show patient name and specialty for each appointment.**

In [85]:
pd.read_sql("""
SELECT p.name, a.specialty
FROM patients p
JOIN appointments a
ON p.patient_id = a.patient_id;

""", conn)


,name,specialty
0,Isha,Dermatology
1,Isha,General
2,Aman,Dental
3,Aman,General
4,Aman,Orthopedic
5,Riya,Dental
6,Riya,Dental
7,Kabir,Cardiology
8,Kabir,Cardiology
9,Neha,ENT


**Show total amount spent by each patient.**

In [86]:
pd.read_sql("""
SELECT p.name, SUM(a.fee) AS total_spent
FROM patients p
JOIN appointments a
ON p.patient_id = a.patient_id
GROUP BY p.name;

""", conn)


,name,total_spent
0,Aman,3050
1,Isha,1700
2,Kabir,5200
3,Neha,1200
4,Pallavi,400
5,Riya,1700
6,Rohit,1800
7,Saurav,1800


**Show total revenue generated from each city.**

In [87]:
pd.read_sql("""
SELECT p.city, SUM(a.fee) AS total_revenue
FROM patients p
JOIN appointments a
ON p.patient_id = a.patient_id
GROUP BY p.city;

""", conn)


,city,total_revenue
0,Bangalore,1200
1,Chennai,1800
2,Delhi,6900
3,Mumbai,3450
4,Pune,3500


**Show number of appointments per specialty.**

In [88]:
pd.read_sql("""
SELECT specialty, COUNT(*) AS total_appointments
FROM appointments
GROUP BY specialty;

""", conn)


,specialty,total_appointments
0,Cardiology,2
1,Dental,3
2,Dermatology,2
3,ENT,1
4,General,5
5,Orthopedic,2


# 🟠 LEVEL 3 — CASE + Reasoning

**For each appointment, show fee and label it:
"Expensive" if fee ≥ 1500
"Affordable" otherwise**

In [89]:
pd.read_sql("""
SELECT appointment_id, fee,
       CASE
         WHEN fee >= 1500 THEN 'Expensive'
         ELSE 'Affordable'
       END AS fee_label
FROM appointments;

""", conn)


,appointment_id,fee,fee_label
0,501,500,Affordable
1,502,1200,Affordable
2,503,1500,Expensive
3,504,600,Affordable
4,505,800,Affordable
5,506,900,Affordable
6,507,2500,Expensive
7,508,500,Affordable
8,509,700,Affordable
9,510,1800,Expensive


**For each patient, show total spending and label:
"High Value" if total ≥ 3000
"Regular" otherwise**

In [90]:
pd.read_sql("""
SELECT p.name,
       SUM(a.fee) AS total_spent,
       CASE
         WHEN SUM(a.fee) >= 3000 THEN 'High Value'
         ELSE 'Regular'
       END AS patient_type
FROM patients p
JOIN appointments a
ON p.patient_id = a.patient_id
GROUP BY p.name;

""", conn)


,name,total_spent,patient_type
0,Aman,3050,High Value
1,Isha,1700,Regular
2,Kabir,5200,High Value
3,Neha,1200,Regular
4,Pallavi,400,Regular
5,Riya,1700,Regular
6,Rohit,1800,Regular
7,Saurav,1800,Regular


**For each city, show total revenue and label:
"Strong City" if revenue ≥ 4000
"Growing City" otherwise**

In [91]:
pd.read_sql("""
SELECT p.city,
       SUM(a.fee) AS total_revenue,
       CASE
         WHEN SUM(a.fee) >= 4000 THEN 'Strong City'
         ELSE 'Growing City'
       END AS city_status
FROM patients p
JOIN appointments a
ON p.patient_id = a.patient_id
GROUP BY p.city;

""", conn)


,city,total_revenue,city_status
0,Bangalore,1200,Growing City
1,Chennai,1800,Growing City
2,Delhi,6900,Strong City
3,Mumbai,3450,Growing City
4,Pune,3500,Growing City


# 🔵 LEVEL 4 — Subqueries (Analytical thinking)

**Find appointments where fee is greater than the average fee.**

In [92]:
pd.read_sql("""
SELECT *
FROM appointments
WHERE fee > (
  SELECT AVG(fee)
  FROM appointments
);

""", conn)


,appointment_id,patient_id,specialty,fee,appointment_date
0,502,1,Dermatology,1200,2024-02-10
1,503,2,Orthopedic,1500,2024-01-12
2,507,4,Cardiology,2500,2024-03-01
3,510,6,Orthopedic,1800,2024-03-15
4,512,8,Dermatology,1300,2024-02-22
5,514,4,Cardiology,2700,2024-04-01


**Find patients whose total spending is greater than average patient spending.**

In [93]:
pd.read_sql("""
SELECT patient_id
FROM appointments
GROUP BY patient_id
HAVING SUM(fee) >
(
  SELECT AVG(total_spent)
  FROM (
    SELECT SUM(fee) AS total_spent
    FROM appointments
    GROUP BY patient_id
  )
);

""", conn)


,patient_id
0,2
1,4


**Find the highest appointment fee (without ORDER BY LIMIT).**

In [94]:
pd.read_sql("""
SELECT *
FROM appointments
WHERE fee = (
  SELECT MAX(fee)
  FROM appointments
);

""", conn)


,appointment_id,patient_id,specialty,fee,appointment_date
0,514,4,Cardiology,2700,2024-04-01


**Find specialties whose total revenue is greater than average specialty revenue.**

In [95]:
pd.read_sql("""
SELECT specialty
FROM appointments
GROUP BY specialty
HAVING SUM(fee) >
(
  SELECT AVG(total_rev)
  FROM (
    SELECT SUM(fee) AS total_rev
    FROM appointments
    GROUP BY specialty
  )
);

""", conn)


,specialty
0,Cardiology
1,Orthopedic


# 🟣 LEVEL 5 — Window Functions

**Show each appointment with the running total of spending per patient.**

In [96]:
pd.read_sql("""
SELECT patient_id, appointment_date, fee,
       SUM(fee) OVER (
         PARTITION BY patient_id
         ORDER BY appointment_date
       ) AS running_total
FROM appointments;

""", conn)


,patient_id,appointment_date,fee,running_total
0,1,2024-01-05,500,500
1,1,2024-02-10,1200,1700
2,2,2024-01-12,1500,1500
3,2,2024-03-01,600,2100
4,2,2024-04-10,950,3050
5,3,2024-02-03,800,800
6,3,2024-03-20,900,1700
7,4,2024-03-01,2500,2500
8,4,2024-04-01,2700,5200
9,5,2024-01-10,500,500


**Rank patients by total spending (highest spender = rank 1).**

In [97]:
pd.read_sql("""
SELECT patient_id, total_spent,
       RANK() OVER (ORDER BY total_spent DESC) AS rank
FROM (
  SELECT patient_id, SUM(fee) AS total_spent
  FROM appointments
  GROUP BY patient_id
);

""", conn)


,patient_id,total_spent,rank
0,4,5200,1
1,2,3050,2
2,6,1800,3
3,8,1800,3
4,1,1700,5
5,3,1700,5
6,5,1200,7
7,7,400,8


**Show each appointment and also show overall average fee (using window function).**

In [98]:
pd.read_sql("""
SELECT appointment_id, fee,
       AVG(fee) OVER () AS overall_avg_fee
FROM appointments;

""", conn)


,appointment_id,fee,overall_avg_fee
0,501,500,1123.333333
1,502,1200,1123.333333
2,503,1500,1123.333333
3,504,600,1123.333333
4,505,800,1123.333333
5,506,900,1123.333333
6,507,2500,1123.333333
7,508,500,1123.333333
8,509,700,1123.333333
9,510,1800,1123.333333


**For each city, rank patients by total spending within their city.**

In [99]:
pd.read_sql("""
SELECT city, name, total_spent,
       RANK() OVER (
         PARTITION BY city
         ORDER BY total_spent DESC
       ) AS city_rank
FROM (
  SELECT p.city, p.name, SUM(a.fee) AS total_spent
  FROM patients p
  JOIN appointments a
  ON p.patient_id = a.patient_id
  GROUP BY p.city, p.name
);

""", conn)


,city,name,total_spent,city_rank
0,Bangalore,Neha,1200,1
1,Chennai,Saurav,1800,1
2,Delhi,Kabir,5200,1
3,Delhi,Isha,1700,2
4,Mumbai,Aman,3050,1
5,Mumbai,Pallavi,400,2
6,Pune,Rohit,1800,1
7,Pune,Riya,1700,2


**Show top 2 highest spending patients using window function.**

In [100]:
pd.read_sql("""
SELECT *
FROM (
  SELECT patient_id,
         SUM(fee) AS total_spent,
         RANK() OVER (ORDER BY SUM(fee) DESC) AS rnk
  FROM appointments
  GROUP BY patient_id
)
WHERE rnk <= 2;

""", conn)


,patient_id,total_spent,rnk
0,4,5200,1
1,2,3050,2
